In [ ]:
#Loading in Packages and Data

#Importing Packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import matplotlib.lines as mlines
import xarray as xr
import os; import time
import pickle
import h5py
###############################################################
def coefs(coefficients,degree):
    coef=coefficients
    coefs=""
    for n in range(degree, -1, -1):
        string=f"({coefficients[len(coef)-(n+1)]:.1e})"
        coefs+=string + f"x^{n}"
        if n != 0:
            coefs+=" + "
    return coefs
###############################################################
start_time = time.time();

#Importing Model Data
check=False
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
job_array=False;index_adjust=0
ocean_fraction=2/8

# dx = 1 km; Np = 1M; Nt = 5 min
data1=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_1km_5min.nc', decode_timedelta=True) #***
parcel1=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_pdata_1km_5min_1e6.nc', decode_timedelta=True) #***
res='1km';t_res='5min'
Np_str='1e6'

# # dx = 1km; Np = 50M
# #Importing Model Data
# dir2='/home/air673/koa_scratch/'
# data1=xr.open_dataset(dir2+'cm1out_1km_1min.nc', decode_timedelta=True) #***
# parcel1=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_50M.nc', decode_timedelta=True) #***
# res='1km'; t_res='1min'; Np_str='50e6'

# # dx = 1km; Np = 50M; Nz = 95
# #Importing Model Data
# dir2='/home/air673/koa_scratch/'
# data1=xr.open_dataset(dir2+'cm1out_1km_1min_95nz.nc', decode_timedelta=True) #***
# parcel1=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_95nz.nc', decode_timedelta=True) #***
# res='1km'; t_res='1min_95nz'; Np_str='50e6'

# # dx = 250m; Np = 50M
# #Importing Model Data
# dir2='/home/air673/koa_scratch/'
# data1=xr.open_dataset(dir2+'cm1out_250m_1min_50M.nc', decode_timedelta=True) #***
# parcel1=xr.open_dataset(dir2+'cm1out_pdata_250m_1min_50M.nc', decode_timedelta=True) #***
# res='250m'; t_res='1min'; Np_str='50e6'

In [ ]:
############################################################################################
#MODEL AND ALGORITHM NUMERICAL PARAMETERS
times=data1['time'].values/(1e9 * 60); times=times.astype(float);
minutes=1/times[1] #1 / minutes per timestep = timesteps per minute
kms=np.argmax(data1['xh'].values-data1['xh'][0].values >= 1) #finds how many x grids is 1 km

In [ ]:
#Import PlottingFunctions
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
path=dir2+'../Functions/'
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions


# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

In [ ]:
###################################################################################################################################
#PLOTTING
plotting=False #KEEP FALSE WHEN RUNNING
plotting=True

In [ ]:
#########################################
#LOADING DATA

In [ ]:
def get_conv(t,z):
    import h5py
    # print('calculating convergence and taking mean')
    if res=='1km':
        dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
    elif res=='250m':
        dir2='/home/air673/koa_scratch/'
    
    file_path = dir2 + 'Variable_Calculation/OUTPUT/' + 'Convergence' + f'_{res}_{t_res}' + '.h5'
    with h5py.File(file_path, 'r') as f:
        Conv = f['conv'][t,z]
    return Conv

def LoadTrackedData(ONLY_SBZ):
    # Define the path to your output file
    ONLY_SBZ = False  # or True
    varname = 'ONLY_SBZS' if ONLY_SBZ else 'ALL_CLS'
    
    # File path
    file_path = f'/mnt/lustre/koa/koastore/torri_group/air_directory/' \
                f'DCI-Project/Project_Algorithms/Tracking_Algorithms/' \
                f'CL_Tracking_Out/' \
                f'whereCL_{res}_{t_res}_{varname}.h5'
    
    # Open dataset (as it's valid NetCDF)
    whereCL = xr.open_dataset(file_path, engine='netcdf4')['maxconv_x'] #chunks=dict(time=1, z=-1, y=-1,x=-1)
    return whereCL

ONLY_SBZ=False
if ONLY_SBZ==True:
    whereCL_ONLY_SBZS=LoadTrackedData(ONLY_SBZ)
elif ONLY_SBZ==False:
    whereCL_ALL_CLS=LoadTrackedData(ONLY_SBZ)

In [ ]:
####PLOT FORM /FIGURES.ipynb
def CL_plotting(data1, t, zlev, ax, ONLY_SBZ, font_size=12, index_adjust=0):
    if np.mod(t, 10) == 0:
        print(f'Current time step: {t}/{len(data1["time"])}')

    print('Loading Data')
    conv_z = get_conv(t, zlev)
    print('Plotting')

    dx = np.round(data1['xf'][1] - data1['xf'][0], 2).item()
    dy = dx

    xh = data1['xh'] - data1['xh'][0]
    yh = data1['yh'] - data1['yh'][0]

    contour = ax.contourf(xh, yh, conv_z * 1000, levels=40)
    cbar = plt.colorbar(contour, ax=ax, pad=0)
    # cbar.set_label(f'{dx * 1000:.0f}' + r'$\ s^{-1}$', fontsize=font_size)
    # cbar.set_label(r'$-\nabla \cdot \mathbf{V}_H\ (s^{-1}$)', fontsize=font_size)
    cbar.set_label(r'$-\nabla \cdot \vec{V}_H\ (s^{-1})$', fontsize=font_size)
    cbar.ax.tick_params(labelsize=font_size)
    cbar.ax.yaxis.label.set_size(font_size)

    ([days, hours, mins], _) = get_time(data1, t + index_adjust, (0, 6, 0))
    z_value = data1['zf'][zlev].values * 1000

    # ax.set_title(f'Horizontal Convergence at t = {t + index_adjust} = {days}:{hours}:{mins}, z = {z_value:.0f} m\nWith Tracked Convergence Local Y-Maxima Overlayed',fontsize=font_size)
    ax.set_xlabel('x (km)', fontsize=font_size)
    ax.set_ylabel('y (km)', fontsize=font_size)
    ax.tick_params(axis='both', which='major', labelsize=font_size)

    # Scatter max convergence points
    for yind in range(len(data1['yh'])):
        if ONLY_SBZ:
            local_maxes = whereCL_ONLY_SBZS[t, zlev, yind].data
        else:
            local_maxes = whereCL_ALL_CLS[t, zlev, yind].data
        local_maxes = local_maxes[local_maxes != -1]
        local_maxes = local_maxes.astype(int)
        # ax.scatter(local_maxes, [yind] * len(local_maxes), color='red', s=1)
        ax.scatter(xh[local_maxes], [yh[yind]] * len(local_maxes), color='red', s=1)


    # Coastline
    ax.axvline(x=len(data1['xf']) * ocean_fraction, color='black', linewidth=1.5, label='Coastline')

    # Legend
    handle_pts = mlines.Line2D([], [], color='red', marker='o', linestyle='None', markersize=6, label='Convergence Local Y-Maxima')
    handle_time = mlines.Line2D([], [], color='none', label=f't = {t + index_adjust} = {days}:{hours}:{mins}')
    handle_z = mlines.Line2D([], [], color='none', label=f'z = {data1["zh"][zlev].item() * 1000:.0f} m')
    handle_c = mlines.Line2D([], [], color='black', lw=3, label='Coastline')
    legend = ax.legend(handles=[handle_pts, handle_c, handle_time, handle_z], loc='upper left', fontsize=font_size)
    for text in legend.get_texts():
        text.set_fontsize(font_size)

def RunTrackedPlot(t,zlev,ONLY_SBZ=False,SAVING=False,CLOSE=False):
    channel_aspect_ratio = 5
    figwidth = 20
    dpi = 72

    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(figwidth, figwidth / channel_aspect_ratio), dpi=dpi)

    CL_plotting(data1, t, zlev, ax, ONLY_SBZ=ONLY_SBZ)

    # SAVING PLOT
    if SAVING:
        folder_path= '/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/Project_Algorithms/Tracking_Algorithms/CL_Tracking_Out/Plots/'
        subfolder = 'ALL_CLS' if not ONLY_SBZ else 'ONLY_SBZS'
        save_path = os.path.join(folder_path, subfolder)
        os.makedirs(save_path, exist_ok=True)

        filename = f'plot_{t + index_adjust}.png'
        fig.savefig(os.path.join(save_path, filename), dpi=dpi)

    if CLOSE==True:
        plt.close(fig)


In [ ]:
if plotting==True:
    #TESTING INDIVIDUAL PLOTS
    #########################
    t = 100 if t_res == '5min' else 100 * 5
    # t+=20
    if (res=='1km' and t_res=='5min') or (res=='1km' and t_res=='1min'):
        zlev=3
    else: 
        zlev=7
    RunTrackedPlot(t=t,zlev=zlev,ONLY_SBZ=False,SAVING=False,CLOSE=False)

In [ ]:
if plotting==True:
    #OUTPUTTING ALL PLOTS
    #########################
    if (res=='1km' and t_res=='5min') or (res=='1km' and t_res=='1min'):
        zlev=3
    else: 
        zlev=7

    for t in np.arange(len(data1['time'])):
        RunTrackedPlot(t=t,zlev=zlev,ONLY_SBZ=False,SAVING=True,CLOSE=True)

In [ ]:
########################################
#MAKING ANIMATION

In [ ]:
if plotting==True:
    #MAKE A GIF OF ALL TRACKED IMAGES
    #####################
    from PIL import Image
    import os
    
    ONLY_SBZ=False
    # ONLY_SBZ=True
    # Folder containing the pictures
    if ONLY_SBZ==False:
        input_folder = '/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/Project_Algorithms/Tracking_Algorithms/CL_Tracking_Out/Plots/ALL_CLS/'
    if ONLY_SBZ==True:
        input_folder = '/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/Project_Algorithms/Tracking_Algorithms/CL_Tracking_Out/Plots/ONLY_SBZS/'
    
    image_filenames = sorted([filename for filename in os.listdir(input_folder) if filename.endswith((".jpg", ".png"))], 
                             key=lambda x: int(x.split('_')[-1].split('.')[0]))
    
    # List to store image objects
    images = []
    
    # Iterate over all files in the folder
    for filename in image_filenames:
        # Check if the file is an image
        if (filename.endswith(".jpg") or filename.endswith(".png")):
            # Open the image and append it to the list
            images.append(Image.open(os.path.join(input_folder, filename)))
    
    # Save images as a GIF
    if ONLY_SBZ==False:
        output_path = input_folder+f'CL_plots_{res}_{t_res}_ALL_CLS.gif'
    if ONLY_SBZ==True:
        output_path = input_folder+f'CL_plots_{res}_{t_res}_ONLY_SBZS.gif'
    images[0].save(output_path,
                   save_all=True,
                   append_images=images[1:],
                   duration=100,  # Specify duration for each frame in milliseconds
                   loop=0)

In [ ]:
if plotting==True:
    # mamba install -c conda-forge moviepy
    # mamba install -c conda-forge ffmpeg
    def convert_gif_to_mp4(input_file, output_file, fps,speed,bitrate='750k'):
        from moviepy.editor import VideoFileClip, vfx
        # Load the GIF file
        gif_clip = VideoFileClip(input_file)
    
        # Set the desired framerate if provided
        if fps:
            gif_clip = gif_clip.set_fps(fps)
        if speed != 1.0:
            gif_clip = gif_clip.fx(vfx.speedx, speed)
    
        # Write the GIF as an MP4 file
        gif_clip.write_videofile(output_file, codec="libx264",bitrate=bitrate)
    
    ONLY_SBZ=False
    # ONLY_SBZ=True
    input_folder = '/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/Project_Algorithms/Tracking_Algorithms/CL_Tracking_Out/Plots/'
    if ONLY_SBZ==False:
        input_folder += 'ALL_CLS/'
    elif ONLY_SBZ==True:
        input_folder += 'ONLY_SBZS/'
    input_file = input_folder+f'CL_plots_{res}_{t_res}_ALL_CLS.gif'
    
    if ONLY_SBZ==False:
        output_file = f'/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/Project_Algorithms/Tracking_Algorithms/CL_Tracking_Out/CL_plots_{res}_{t_res}_ALL_CLS.mp4'
    if ONLY_SBZ==True:
            output_file = f'/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/Project_Algorithms/Tracking_Algorithms/CL_Tracking_Out/CL_plots_{res}_{t_res}_ONLY_SBZS.mp4'
        
    convert_gif_to_mp4(input_file, output_file, speed=0.25,fps=None)